In [5]:
import shap
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from itertools import combinations
import seaborn as sns
from sklearn import metrics
from sklearn import preprocessing 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.feature_selection import RFE
from bayes_opt import BayesianOptimization

In [2]:
warnings.filterwarnings('ignore')

def run_bayesian_optimization(X_train_all, y_train_all):
    def black_box_function(learning_rate, n_estimators, min_samples_split, max_features, max_depth, max_leaf_nodes):
        params = {
            'learning_rate': max(learning_rate, 1e-3),
            'n_estimators': int(n_estimators),
            'min_samples_split': int(min_samples_split),
            'max_features': min(max_features, 0.999),
            'max_depth': int(max_depth),
            'max_leaf_nodes': int(max_leaf_nodes),
            'random_state': 2
        }
        
        model = GradientBoostingRegressor(**params)
        loo = LeaveOneOut()
        preds, truths = [], []
        
        for train_idx, val_idx in loo.split(X_train_all):
            X_train, X_val = X_train_all[train_idx], X_train_all[val_idx]
            y_train, y_val = y_train_all[train_idx], y_train_all[val_idx]
            model.fit(X_train, y_train.ravel())
            preds.append(model.predict(X_val)[0])
            truths.append(y_val[0])
            
        return r2_score(truths, preds)

    pbounds = {
        'learning_rate': (0.001, 0.2),
        'n_estimators': (10, 500),
        'min_samples_split': (2, 25),
        'max_features': (0.1, 1.0),
        'max_depth': (1, 5),
        'max_leaf_nodes': (2, 15)
    }

    optimizer = BayesianOptimization(
        f=black_box_function,
        pbounds=pbounds,
        random_state=1
    )
    optimizer.maximize(init_points=15, n_iter=20)
    return optimizer.max['params']


data = pd.read_excel(r"C:\Users\HP\Desktop\Data.xlsx", 
                    sheet_name='16+3',
                    index_col=0,
                    header=0)
features = data[['lg(O3)', 'lg(H2O2)', 'pH', 'TOC']]
data1 = features.iloc[0:18]

r2_results = []
rmse_results = []

for iter in range(10):
    print(f"\n========== Iteration times {iter+1}/10 ==========")
    
    test_indices = np.random.choice(data1.index, size=4, replace=False)
    test_set = data1.loc[test_indices]
    train_set = data1.drop(test_indices)
    
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(train_set[['lg(O3)', 'lg(H2O2)', 'pH']])
    y_train = train_set['TOC'].values.reshape(-1, 1)
    
    train_df = pd.DataFrame(X_train, 
                           columns=['lg(O3)', 'lg(H2O2)', 'pH'],
                           index=train_set.index)
    train_df['TOC'] = y_train
    
    noise_ratio = 0.08
    num_new_samples = 36
    noise_columns = ['lg(O3)', 'lg(H2O2)', 'pH']
    
    np.random.seed(iter)  
    
    enhanced_data = []
    for _ in range(num_new_samples):

        idx = np.random.randint(0, len(train_df))
        sample = train_df.iloc[idx].copy()
        

        for col in noise_columns:
            original_value = sample[col]
            if original_value != 0:
                std_dev = abs(original_value) * noise_ratio
                sample[col] = np.random.normal(loc=original_value, scale=std_dev)
        
        enhanced_data.append(sample)
    
    augmented = pd.DataFrame(enhanced_data)
    X_augmented = np.vstack([X_train, augmented[noise_columns].values])
    y_augmented = np.vstack([y_train, augmented['TOC'].values.reshape(-1, 1)])
    
    best_params = run_bayesian_optimization(X_augmented, y_augmented)
    
    final_model = GradientBoostingRegressor(
        learning_rate=best_params['learning_rate'],
        n_estimators=int(best_params['n_estimators']),
        min_samples_split=int(best_params['min_samples_split']),
        max_features=best_params['max_features'],
        max_depth=int(best_params['max_depth']),
        max_leaf_nodes=int(best_params['max_leaf_nodes']),
        random_state=2
    )
    final_model.fit(X_augmented, y_augmented.ravel())
    
    X_test = scaler.transform(test_set[['lg(O3)', 'lg(H2O2)', 'pH']])
    y_test = test_set['TOC'].values
    y_pred = final_model.predict(X_test)
    
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    r2_results.append(r2)
    rmse_results.append(rmse)
    print(f"Results of this round: R2={r2:.4f}, RMSE={rmse:.4f}")

print("\n================ Final result ================")
print(f"Average R2 score: {np.mean(r2_results):.4f} ± {np.std(r2_results):.4f}")
print(f"Average RMSE score: {np.mean(rmse_results):.4f} ± {np.std(rmse_results):.4f}")
print("\n Detailed R2 results:", [round(x, 4) for x in r2_results])
print("Detailed RMSE results:", [round(x, 4) for x in rmse_results])


========== 迭代次数 1/10 ==========
|   iter    |  target   | learni... | max_depth | max_fe... | max_le... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------------------
| 1         | 0.958     | 0.08399   | 3.881     | 0.1001    | 5.93      | 5.375     | 55.25     |
| 2         | 0.9588    | 0.03807   | 2.382     | 0.4571    | 9.005     | 11.64     | 345.8     |
| 3         | 0.9543    | 0.04169   | 4.512     | 0.1246    | 10.72     | 11.6      | 283.8     |
| 4         | 0.9311    | 0.02894   | 1.792     | 0.8207    | 14.59     | 9.209     | 349.2     |
| 5         | 0.9391    | 0.1754    | 4.578     | 0.1765    | 2.508     | 5.906     | 440.3     |
| 6         | 0.917     | 0.02057   | 2.684     | 0.9621    | 8.931     | 17.91     | 164.6     |
| 7         | 0.9539    | 0.1376    | 4.339     | 0.1165    | 11.75     | 24.74     | 376.6     |
| 8         | 0.9671    | 0.05681   | 4.157     | 0.1929    | 7.823     | 22.9      |

| 9         | 0.8761    | 0.05827   | 1.52      | 0.1174    | 10.82     | 6.867     | 140.1     |
| 10        | 0.906     | 0.09882   | 1.213     | 0.6167    | 3.907     | 15.55     | 352.9     |
| 11        | 0.9278    | 0.02136   | 2.656     | 0.725     | 7.384     | 3.149     | 272.6     |
| 12        | 0.9127    | 0.1331    | 3.06      | 0.9501    | 9.625     | 22.78     | 77.36     |
| 13        | 0.9257    | 0.02872   | 4.23      | 0.4579    | 4.15      | 23.33     | 180.4     |
| 14        | 0.9028    | 0.1504    | 3.904     | 0.895     | 10.11     | 19.27     | 181.0     |
| 15        | 0.8987    | 0.05472   | 4.584     | 0.4853    | 14.54     | 17.26     | 314.6     |
| 16        | 0.9079    | 0.08639   | 2.721     | 0.79      | 7.449     | 3.214     | 272.7     |
| 17        | 0.9032    | 0.1777    | 2.789     | 0.6977    | 5.599     | 16.38     | 429.7     |
| 18        | 0.9165    | 0.1118    | 1.112     | 0.9497    | 14.44     | 19.54     | 438.5     |
| 19        | 0.8538

| 19        | 0.9309    | 0.1334    | 4.157     | 0.1839    | 10.49     | 11.69     | 283.9     |
| 20        | 0.3231    | 0.001     | 4.378     | 0.1       | 10.74     | 11.99     | 284.2     |
| 21        | 0.3263    | 0.002686  | 2.355     | 0.6387    | 2.712     | 16.91     | 229.1     |
| 22        | 0.9318    | 0.05254   | 4.857     | 0.2515    | 12.21     | 12.49     | 115.3     |
| 23        | 0.9076    | 0.06679   | 1.646     | 0.3375    | 9.999     | 10.33     | 488.5     |
| 24        | 0.9097    | 0.02503   | 2.029     | 0.2521    | 14.07     | 14.75     | 163.4     |
| 25        | 0.936     | 0.07754   | 3.498     | 0.5247    | 8.575     | 20.35     | 485.0     |
| 26        | 0.926     | 0.2       | 4.344     | 0.399     | 10.58     | 11.54     | 283.7     |
| 27        | 0.9384    | 0.0481    | 4.519     | 0.1311    | 10.72     | 11.6      | 283.8     |
| 28        | 0.9143    | 0.2       | 3.831     | 0.2155    | 10.2      | 11.48     | 283.6     |
| 29        | 0.9082

| 29        | 0.8024    | 0.0739    | 1.727     | 0.8856    | 9.309     | 16.39     | 458.7     |
| 30        | 0.7973    | 0.1186    | 1.18      | 0.6864    | 10.74     | 4.78      | 217.8     |
| 31        | 0.6503    | 0.004293  | 3.756     | 0.2024    | 12.86     | 3.325     | 234.5     |
| 32        | 0.8002    | 0.1639    | 1.364     | 0.2557    | 4.134     | 24.37     | 234.2     |
| 33        | 0.848     | 0.1624    | 4.524     | 0.416     | 14.96     | 5.073     | 309.8     |
| 34        | 0.8359    | 0.1552    | 3.482     | 0.1798    | 11.12     | 19.23     | 60.62     |
| 35        | 0.7793    | 0.1779    | 1.378     | 0.1688    | 13.33     | 18.66     | 445.5     |
本轮结果: R2=0.8311, RMSE=0.0369

========== 迭代次数 5/10 ==========
|   iter    |  target   | learni... | max_depth | max_fe... | max_le... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------------------
| 1         | 0.8488    | 0.08399   | 3.881     | 0.1001

| 2         | 0.9626    | 0.03807   | 2.382     | 0.4571    | 9.005     | 11.64     | 345.8     |
| 3         | 0.9612    | 0.04169   | 4.512     | 0.1246    | 10.72     | 11.6      | 283.8     |
| 4         | 0.9195    | 0.02894   | 1.792     | 0.8207    | 14.59     | 9.209     | 349.2     |
| 5         | 0.9286    | 0.1754    | 4.578     | 0.1765    | 2.508     | 5.906     | 440.3     |
| 6         | 0.9212    | 0.02057   | 2.684     | 0.9621    | 8.931     | 17.91     | 164.6     |
| 7         | 0.9772    | 0.1376    | 4.339     | 0.1165    | 11.75     | 24.74     | 376.6     |
| 8         | 0.9692    | 0.05681   | 4.157     | 0.1929    | 7.823     | 22.9      | 153.9     |
| 9         | 0.9121    | 0.05827   | 1.52      | 0.1174    | 10.82     | 6.867     | 140.1     |
| 10        | 0.931     | 0.09882   | 1.213     | 0.6167    | 3.907     | 15.55     | 352.9     |
| 11        | 0.9481    | 0.02136   | 2.656     | 0.725     | 7.384     | 3.149     | 272.6     |
| 12        | 0.9808

| 12        | 0.9127    | 0.1331    | 3.06      | 0.9501    | 9.625     | 22.78     | 77.36     |
| 13        | 0.9363    | 0.02872   | 4.23      | 0.4579    | 4.15      | 23.33     | 180.4     |
| 14        | 0.8985    | 0.1504    | 3.904     | 0.895     | 10.11     | 19.27     | 181.0     |
| 15        | 0.9248    | 0.05472   | 4.584     | 0.4853    | 14.54     | 17.26     | 314.6     |
| 16        | 0.9238    | 0.1035    | 4.304     | 0.5344    | 4.222     | 23.41     | 180.5     |
| 17        | 0.8822    | 0.1777    | 2.789     | 0.6977    | 5.599     | 16.38     | 429.7     |
| 18        | 0.1952    | 0.001     | 4.117     | 0.3417    | 4.04      | 23.22     | 180.3     |
| 19        | 0.8879    | 0.03792   | 3.435     | 0.5635    | 5.672     | 3.495     | 46.19     |
| 20        | 0.912     | 0.07941   | 3.735     | 0.2562    | 6.286     | 9.344     | 212.0     |
| 21        | 0.4675    | 0.002686  | 2.355     | 0.6387    | 2.712     | 16.91     | 229.1     |
| 22        | 0.9247

| 22        | 0.9578    | 0.05254   | 4.857     | 0.2515    | 12.21     | 12.49     | 115.3     |
| 23        | 0.9551    | 0.06679   | 1.646     | 0.3375    | 9.999     | 10.33     | 488.5     |
| 24        | 0.9375    | 0.02503   | 2.029     | 0.2521    | 14.07     | 14.75     | 163.4     |
| 25        | 0.9687    | 0.07754   | 3.498     | 0.5247    | 8.575     | 20.35     | 485.0     |
| 26        | 0.9712    | 0.1211    | 2.986     | 0.7659    | 5.591     | 16.49     | 429.7     |
| 27        | 0.9791    | 0.1562    | 3.911     | 0.9017    | 10.11     | 19.28     | 181.0     |
| 28        | 0.9739    | 0.2       | 3.689     | 1.0       | 10.07     | 19.17     | 180.7     |
| 29        | 0.966     | 0.0739    | 1.727     | 0.8856    | 9.309     | 16.39     | 458.7     |
| 30        | 0.9587    | 0.03523   | 4.155     | 0.2296    | 8.172     | 22.7      | 153.8     |
| 31        | 0.9675    | 0.0748    | 3.369     | 0.5566    | 8.786     | 20.58     | 485.2     |
| 32        | 0.9768

| 32        | 0.8511    | 0.1639    | 1.364     | 0.2557    | 4.134     | 24.37     | 234.2     |
| 33        | 0.8923    | 0.1624    | 4.524     | 0.416     | 14.96     | 5.073     | 309.8     |
| 34        | 0.8998    | 0.2       | 2.694     | 0.7928    | 7.388     | 3.184     | 272.6     |
| 35        | 0.8349    | 0.1779    | 1.378     | 0.1688    | 13.33     | 18.66     | 445.5     |
本轮结果: R2=0.2850, RMSE=0.1501

========== 迭代次数 10/10 ==========
|   iter    |  target   | learni... | max_depth | max_fe... | max_le... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------------------
| 1         | 0.8746    | 0.08399   | 3.881     | 0.1001    | 5.93      | 5.375     | 55.25     |
| 2         | 0.9226    | 0.03807   | 2.382     | 0.4571    | 9.005     | 11.64     | 345.8     |
| 3         | 0.9243    | 0.04169   | 4.512     | 0.1246    | 10.72     | 11.6      | 283.8     |
| 4         | 0.919     | 0.02894   | 1.792     | 0.820